In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [79]:
df1 = pd.read_csv('eda/small1/labeled.csv')
df2 = pd.read_csv('eda/small2/labeled.csv')
df3 = pd.read_csv('eda/small3/labeled.csv')
df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
df.to_csv('data/merged_titles_labels.csv', index=False)
df.head()

,title,is_clickbait
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [80]:
%run functions.py
df = preprocess_title(df)
df.head()

# save preprocessed data as pickle file
df.to_pickle('data/preprocessed_titles_labels.pkl')

In [74]:
from tqdm.notebook import tqdm
import time
# VECTOR_SIZE = 2500
# WINDOW = 4
EPOCHS = 500

# EPOCHS = 100
WORKERS = 19
MIN_COUNT = 1
# train many word2vec models with diferent VECTOR_SIZE and WINDOW
VECTOR_SIZEs = [
    100, 
    250, 
    500, 
    1000, 
    # 1500, 
    # 2000, 
    # 2500, 
    # 3000
]

# WINDOWs = [3, 4, 5, 6, 7, 8]
# SGs = [0, 1]

EPOCHS = 100
VECTOR_SIZEs = [100]
WINDOWs = [4]
SGs = [0]


print('Start training')
# sleep 200 ms
time.sleep(0.2)

for VECTOR_SIZE in tqdm(VECTOR_SIZEs):
    print(f'Current VECTOR_SIZE: {VECTOR_SIZE}')
    for WINDOW in tqdm(WINDOWs, desc=f'WINDOW'):
        for sg in tqdm(SGs, desc=f'SG'):
            model = Word2Vec(df['title'], vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNT, workers=WORKERS, sg=sg)
            model.train(df['title'], total_examples=len(df['title']), epochs=EPOCHS)
            model.save(f'word2vec_models/word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model')

Start training


  0%|          | 0/1 [00:00<?, ?it/s]

Current VECTOR_SIZE: 100


WINDOW:   0%|          | 0/1 [00:00<?, ?it/s]

SG:   0%|          | 0/1 [00:00<?, ?it/s]

In [75]:
model.wv.most_similar('trump')

[('obama', 0.6025455594062805),
 ('hillari', 0.6009495258331299),
 ('breitbart', 0.5067097544670105),
 ('clinton', 0.48945683240890503),
 ('sander', 0.47507503628730774),
 ('donald', 0.4685015082359314),
 ('gop', 0.4669952094554901),
 ('republican', 0.46046730875968933),
 ('tillerson', 0.4540662467479706),
 ('putin', 0.4527655839920044)]

In [76]:
# save to txt the vocabulary
with open('word2vec_models/vocabulary.txt', 'w') as f:
    for word in model.wv.key_to_index:
        f.write(word + '\n')


# show the vector of a word